# First NoteBook

# 1.1 Import Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import gc
import ctypes
import os
import itertools
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import random
import pprint
import time
import copy
import lightgbm as lgb
import torch
import polars as pl
import optuna

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression,Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_absolute_error,mean_squared_error,accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder, PowerTransformer, RobustScaler, FunctionTransformer
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold, train_test_split
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn import metrics
%matplotlib inline
from random import choice, choices
from functools import reduce, partial
from tqdm import tqdm
from itertools import cycle
from collections import Counter
from scipy import stats
from scipy.stats import skew, kurtosis
from transformers import BertTokenizer
from collections import Counter, defaultdict
from tqdm.autonotebook import tqdm
from math import sqrt
from sklearn import model_selection

def clean_memory():
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()
clean_memory()

# 1.2 Feature Engineering

In [2]:
import pandas as pd

def getEssays(df):
  
    # 'id', 'activity', 'cursor_position', 'text_change' 열만 선택한 DataFrame 복사
    textInputDf = df[['id', 'activity', 'cursor_position', 'text_change']].copy()
    
    # 'activity' 열에서 'Nonproduction'인 행을 제외
    textInputDf = textInputDf[textInputDf.activity != 'Nonproduction']

    # 각 'id'별로 발생한 활동 수를 계산하여 배열로 저장
    valCountsArr = textInputDf['id'].value_counts(sort=False).values

    lastIndex = 0

    # 결과를 저장할 Pandas Series 생성
    essaySeries = pd.Series()

    for index, valCount in enumerate(valCountsArr):

        currTextInput = textInputDf[['activity', 'cursor_position', 'text_change']].iloc[lastIndex : lastIndex + valCount]
        lastIndex += valCount
        essayText = ""

        for Input in currTextInput.values:
            
            # Input[0] = activity
            # Input[2] = cursor_position
            # Input[3] = text_change
            
            if Input[0] == 'Replace':
                # '=>' 문자열을 기준으로 text_change를 분할
                replaceTxt = Input[2].split(' => ')
                
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                continue

                
            # If activity = Paste    
            if Input[0] == 'Paste':
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                continue

                
            # If activity = Remove/Cut
            if Input[0] == 'Remove/Cut':
                # DONT TOUCH
                essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                continue

                
            # If activity = Move...
            if "M" in Input[0]:
                # "Move from to" 텍스트를 제거
                croppedTxt = Input[0][10:]
                
                # ' To '를 기준으로 문자열을 분할
                splitTxt = croppedTxt.split(' To ')
                
                # 문자열을 다시 ', '를 기준으로 분할하여 배열로 저장
                valueArr = [item.split(', ') for item in splitTxt]
                
                # Move from [2, 4] To [5, 7] = (2, 4, 5, 7)
                moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))

                # 같은 위치로 이동하는 경우 건너뛰기
                if moveData[0] != moveData[2]:
                    # 텍스트를 앞으로 이동시키는 경우 (다른 경우)
                    if moveData[0] < moveData[2]:
                        # DONT TOUCH
                        essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                    else:
                        # DONT TOUCH
                        essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
                continue
                         
            # If just input
            # DONT TOUCH
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]

        # 결과 시리즈의 해당 인덱스에 에세이 텍스트를 설정  
        essaySeries[index] = essayText
     
    # 결과 시리즈의 인덱스를 고유한 'id' 값으로 설정
    essaySeries.index =  textInputDf['id'].unique()
    
    return essaySeries

In [3]:
traindf = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv')
train_scores = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv')
testdf = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv')

In [4]:
train_essaysdf = pd.read_csv('/kaggle/input/writing-quality-challenge-constructed-essays/train_essays_fast.csv')

In [5]:
train_essaysdf

,id,essay
0,001519c8,qqqqqqqqq qq qqqqq qq qqqq qqqq. qqqqqq qqq q...
1,0022f953,"qqqq qq qqqqqqqqqqq ? qq qq qqq qqq qqq, qqqqq..."
2,0042269b,qqqqqqqqqqq qq qqqqq qqqqqqqqq qq qqqqqqqqqqq ...
3,0059420b,qq qqqqqqq qqqqqq qqqqqqqqqqqqq qqqq q qqqq qq...
4,0075873a,"qqqqqqqqqqq qq qqq qqqqq qq qqqqqqqqqq, qqq qq..."
...,...,...
2466,ffb8c745,"qq qqqqq'q qqqqqqq, qqq'q qqqqq q qqqq qq..."
2467,ffbef7e5,"qqqq qqqqqq qqqqq qq qqqqq qqqqq, qq qq q qqqq..."
2468,ffccd6fd,qqqqqq qqqq q qqqqqqq qqqqqqqqq qq qqqqqq qqqq...
2469,ffec5b38,"qqqqqqqqqq qqqqqqq, qqqqqq qqqq qqqqq qqqq qqq..."


In [6]:
%%time
test_essays = getEssays(testdf)

CPU times: user 7.06 ms, sys: 423 µs, total: 7.48 ms
Wall time: 15.9 ms


In [7]:
test_essaysdf = pd.DataFrame({'id': test_essays.index, 'essay': test_essays.values})

merged_data = train_essaysdf.merge(train_scores, on='id')

In [8]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2))
X_tokenizer_train = count_vectorizer.fit_transform(merged_data['essay'])
X_tokenizer_test = count_vectorizer.transform(test_essaysdf['essay'])
count_vectorizer.get_feature_names_out() #ADDED
y = merged_data['score']

In [9]:
df_train = pd.DataFrame()
df_test = pd.DataFrame()

In [10]:
X_tokenizer_train = X_tokenizer_train.todense()
X_tokenizer_test = X_tokenizer_test.todense()

for i in range(X_tokenizer_train.shape[1]) : 
    L = list(X_tokenizer_train[:,i])
    li = [int(x) for x in L ]
    df_train[f'feature {i}'] = li
    
for i in range(X_tokenizer_test.shape[1]) : 
    L = list(X_tokenizer_test[:,i])
    li = [int(x) for x in L ]
    df_test[f'feature {i}'] = li    

In [11]:
df_train_index = train_essaysdf['id']
df_test_index = test_essaysdf['id']

# id 열 추가 
df_train.loc[:, 'id'] = df_train_index
df_test.loc[:, 'id'] = df_test_index

In [12]:
# 집계함수 적용 df
train_agg_fe_df = traindf.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
train_agg_fe_df.columns = ['_'.join(x) for x in train_agg_fe_df.columns]
train_agg_fe_df = train_agg_fe_df.add_prefix("tmp_")
train_agg_fe_df.reset_index(inplace=True)

test_agg_fe_df = testdf.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
test_agg_fe_df.columns = ['_'.join(x) for x in test_agg_fe_df.columns]
test_agg_fe_df = test_agg_fe_df.add_prefix("tmp_")
test_agg_fe_df.reset_index(inplace=True)

In [13]:
from collections import defaultdict

class Preprocessor:
    
    def __init__(self, seed):
        self.seed = seed
        
        self.activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
        self.events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 
              'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
        self.text_changes = ['q', ' ', 'NoChange', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']
        self.punctuations = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/',
                        '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']
        self.gaps = [1, 2, 3, 5, 10, 20, 50, 100]
        
        self.idf = defaultdict(float)
#         self.gaps = [1, 2]
    
    def activity_counts(self, df):
        tmp_df = df.groupby('id').agg({'activity': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['activity'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.activities:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'activity_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf

            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf

        return ret


    def event_counts(self, df, colname):
        tmp_df = df.groupby('id').agg({colname: list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df[colname].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.events:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'{colname}_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf
            
            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf

        return ret


    def text_change_counts(self, df):
        tmp_df = df.groupby('id').agg({'text_change': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['text_change'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.text_changes:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'text_change_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf
            
            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf
            
        return ret

    def match_punctuations(self, df):
        tmp_df = df.groupby('id').agg({'down_event': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['down_event'].values):
            cnt = 0
            items = list(Counter(li).items())
            for item in items:
                k, v = item[0], item[1]
                if k in self.punctuations:
                    cnt += v
            ret.append(cnt)
        ret = pd.DataFrame({'punct_cnt': ret})
        return ret


    def get_input_words(self, df):
        tmp_df = df[(~df['text_change'].str.contains('=>'))&(df['text_change'] != 'NoChange')].reset_index(drop=True)
        tmp_df = tmp_df.groupby('id').agg({'text_change': list}).reset_index()
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: ''.join(x))
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: re.findall(r'q+', x))
        tmp_df['input_word_count'] = tmp_df['text_change'].apply(len)
        tmp_df['input_word_length_mean'] = tmp_df['text_change'].apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_max'] = tmp_df['text_change'].apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_std'] = tmp_df['text_change'].apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df.drop(['text_change'], axis=1, inplace=True)
        return tmp_df
    
    def make_feats(self, df):
        
        print("Starting to engineer features")
        
        # initialize features dataframe
        feats = pd.DataFrame({'id': df['id'].unique().tolist()})
        
        # get shifted features
        # time shift
        print("Engineering time data")
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f'up_time_shift{gap}'] = df.groupby('id')['up_time'].shift(gap)
            df[f'action_time_gap{gap}'] = df['down_time'] - df[f'up_time_shift{gap}']
        df.drop(columns=[f'up_time_shift{gap}' for gap in self.gaps], inplace=True)

        # cursor position shift
        print("Engineering cursor position data")
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f'cursor_position_shift{gap}'] = df.groupby('id')['cursor_position'].shift(gap)
            df[f'cursor_position_change{gap}'] = df['cursor_position'] - df[f'cursor_position_shift{gap}']
            df[f'cursor_position_abs_change{gap}'] = np.abs(df[f'cursor_position_change{gap}'])
        df.drop(columns=[f'cursor_position_shift{gap}' for gap in self.gaps], inplace=True)

        # word count shift
        print("Engineering word count data")
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f'word_count_shift{gap}'] = df.groupby('id')['word_count'].shift(gap)
            df[f'word_count_change{gap}'] = df['word_count'] - df[f'word_count_shift{gap}']
            df[f'word_count_abs_change{gap}'] = np.abs(df[f'word_count_change{gap}'])
        df.drop(columns=[f'word_count_shift{gap}' for gap in self.gaps], inplace=True)
        
        # get aggregate statistical features
        print("Engineering statistical summaries for features")
        # [(feature name, [ stat summaries to add ])]
        feats_stat = [
            ('event_id', ['max']),
            ('up_time', ['max']),
            ('action_time', ['max', 'min', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew']),
            ('activity', ['nunique']),
            ('down_event', ['nunique']),
            ('up_event', ['nunique']),
            ('text_change', ['nunique']),
            ('cursor_position', ['nunique', 'max', 'quantile', 'sem', 'mean']),
            ('word_count', ['nunique', 'max', 'quantile', 'sem', 'mean'])]
        for gap in self.gaps:
            feats_stat.extend([
                (f'action_time_gap{gap}', ['max', 'min', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew']),
                (f'cursor_position_change{gap}', ['max', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew']),
                (f'word_count_change{gap}', ['max', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew'])
            ])
        
        pbar = tqdm(feats_stat)
        for item in pbar:
            colname, methods = item[0], item[1]
            for method in methods:
                pbar.set_postfix()
                if isinstance(method, str):
                    method_name = method
                else:
                    method_name = method.__name__
                    
                pbar.set_postfix(column=colname, method=method_name)
                tmp_df = df.groupby(['id']).agg({colname: method}).reset_index().rename(columns={colname: f'{colname}_{method_name}'})
                feats = feats.merge(tmp_df, on='id', how='left')

        # counts
        print("Engineering activity counts data")
        tmp_df = self.activity_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering event counts data")
        tmp_df = self.event_counts(df, 'down_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        tmp_df = self.event_counts(df, 'up_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering text change counts data")
        tmp_df = self.text_change_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering punctuation counts data")
        tmp_df = self.match_punctuations(df)
        feats = pd.concat([feats, tmp_df], axis=1)

        # input words
        print("Engineering input words data")
        tmp_df = self.get_input_words(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')

        # compare feats
        print("Engineering ratios data")
        feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
        feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
        feats['event_time_ratio'] = feats['event_id_max']  / feats['up_time_max']
        feats['idle_time_ratio'] = feats['action_time_gap1_sum'] / feats['up_time_max']
        
        print("Done!")
        return feats

In [14]:
preprocessor = Preprocessor(seed=42)

print("Engineering features for training data")

other_train_feats = preprocessor.make_feats(traindf)

print()
print("-"*25)
print("Engineering features for test data")
print("-"*25)
other_test_feats = preprocessor.make_feats(testdf)

Engineering features for training data
Starting to engineer features
Engineering time data
> for gap 1
> for gap 2
> for gap 3
> for gap 5
> for gap 10
> for gap 20
> for gap 50
> for gap 100
Engineering cursor position data
> for gap 1
> for gap 2
> for gap 3
> for gap 5
> for gap 10
> for gap 20
> for gap 50
> for gap 100
Engineering word count data
> for gap 1
> for gap 2
> for gap 3
> for gap 5
> for gap 10
> for gap 20
> for gap 50
> for gap 100
Engineering statistical summaries for features


  0%|          | 0/33 [00:00<?, ?it/s]

Engineering activity counts data


  0%|          | 0/2471 [00:00<?, ?it/s]

Engineering event counts data


...

Engineering text change counts data


  0%|          | 0/3 [00:00<?, ?it/s]

Engineering punctuation counts data


  0%|          | 0/3 [00:00<?, ?it/s]

Engineering input words data
Engineering ratios data
Done!


In [15]:
df_train_all = pd.DataFrame()
df_test_all = pd.DataFrame()

df_train_all = df_train.merge(train_agg_fe_df,on='id')
df_test_all = df_test.merge(test_agg_fe_df,on='id')

In [16]:
def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)

In [17]:
AGGREGATIONS = ['count', 'mean', 'std', 'min', 'max', 'first', 'last', 'sem', q1, 'median', q3, 'skew', 'sum']

def split_essays_into_sentences(df):
    essay_df = df
    essay_df['id'] = essay_df.index
    essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',str(x)))
    essay_df = essay_df.explode('sent')
    essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())
    # Number of characters in sentences
    essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
    # Number of words in sentences
    essay_df['sent_word_count'] = essay_df['sent'].apply(lambda x: len(x.split(' ')))
    essay_df = essay_df[essay_df.columns.tolist()].reset_index(drop=True)
    return essay_df

def compute_sentence_aggregations(df):
    sent_agg_df = pd.concat(
        [df[['id','sent_len']].groupby(['id']).agg(AGGREGATIONS), df[['id','sent_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1
    )
    sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
    sent_agg_df['id'] = sent_agg_df.index
    sent_agg_df = sent_agg_df.reset_index(drop=True)
    sent_agg_df.drop(columns=["sent_word_count_count"], inplace=True)
    sent_agg_df = sent_agg_df.rename(columns={"sent_len_count":"sent_count"})
    return sent_agg_df

def split_essays_into_paragraphs(df):
    essay_df = df
    essay_df['id'] = essay_df.index
    essay_df['paragraph'] = essay_df['essay'].apply(lambda x: str(x).split('\n'))
    essay_df = essay_df.explode('paragraph')
    # Number of characters in paragraphs
    essay_df['paragraph_len'] = essay_df['paragraph'].apply(lambda x: len(x)) 
    # Number of words in paragraphs
    essay_df['paragraph_word_count'] = essay_df['paragraph'].apply(lambda x: len(x.split(' ')))
    essay_df = essay_df[essay_df.paragraph_len!=0].reset_index(drop=True)
    return essay_df

def compute_paragraph_aggregations(df):
    paragraph_agg_df = pd.concat(
        [df[['id','paragraph_len']].groupby(['id']).agg(AGGREGATIONS), df[['id','paragraph_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1
    ) 
    paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
    paragraph_agg_df['id'] = paragraph_agg_df.index
    paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
    paragraph_agg_df.drop(columns=["paragraph_word_count_count"], inplace=True)
    paragraph_agg_df = paragraph_agg_df.rename(columns={"paragraph_len_count":"paragraph_count"})
    return paragraph_agg_df

In [18]:
train_sent_df = split_essays_into_sentences(train_essaysdf)
train_sent_agg_df = compute_sentence_aggregations(train_sent_df)

train_paragraph_df = split_essays_into_paragraphs(train_essaysdf)
train_paragraph_agg_df = compute_paragraph_aggregations(train_paragraph_df)

test_sent_agg_df = compute_sentence_aggregations(split_essays_into_sentences(test_essaysdf))
test_paragraph_agg_df = compute_paragraph_aggregations(split_essays_into_paragraphs(test_essaysdf))

train_paragraph_agg_df.loc[:, 'id'] = df_train_index
train_sent_agg_df.loc[:, 'id'] = df_train_index

test_paragraph_agg_df.loc[:, 'id'] = df_test_index
test_sent_agg_df.loc[:, 'id'] = df_test_index

In [19]:
new_train_feats = pd.DataFrame()
new_test_feats = pd.DataFrame()

new_train_feats = train_paragraph_agg_df.merge(df_train_all,on='id')
new_train_feats = new_train_feats.merge(train_sent_agg_df,on='id')

new_test_feats = test_paragraph_agg_df.merge(df_test_all,on='id')
new_test_feats = new_test_feats.merge(test_sent_agg_df,on='id')

train_feats = pd.DataFrame()
test_feats = pd.DataFrame()

train_feats = new_train_feats.merge(other_train_feats,on='id')
test_feats = new_test_feats.merge(other_test_feats,on='id')

In [20]:
data = []

for logs in [traindf, testdf]:
    logs['up_time_lagged'] = logs.groupby('id')['up_time'].shift(1).fillna(logs['down_time'])
    logs['time_diff'] = abs(logs['down_time'] - logs['up_time_lagged']) / 1000

    group = logs.groupby('id')['time_diff']
    largest_lantency = group.max()
    smallest_lantency = group.min()
    median_lantency = group.median()
    initial_pause = logs.groupby('id')['down_time'].first() / 1000
    pauses_half_sec = group.apply(lambda x: ((x > 0.5) & (x < 1)).sum())
    pauses_1_sec = group.apply(lambda x: ((x > 1) & (x < 1.5)).sum())
    pauses_1_half_sec = group.apply(lambda x: ((x > 1.5) & (x < 2)).sum())
    pauses_2_sec = group.apply(lambda x: ((x > 2) & (x < 3)).sum())
    pauses_3_sec = group.apply(lambda x: (x > 3).sum())

    data.append(pd.DataFrame({
        'id': logs['id'].unique(),
        'largest_lantency': largest_lantency,
        'smallest_lantency': smallest_lantency,
        'median_lantency': median_lantency,
        'initial_pause': initial_pause,
        'pauses_half_sec': pauses_half_sec,
        'pauses_1_sec': pauses_1_sec,
        'pauses_1_half_sec': pauses_1_half_sec,
        'pauses_2_sec': pauses_2_sec,
        'pauses_3_sec': pauses_3_sec,
    }).reset_index(drop=True))

train_eD592674, test_eD592674 = data

train_feats = train_feats.merge(train_eD592674, on='id', how='left')
test_feats = test_feats.merge(test_eD592674, on='id', how='left')
train_feats = train_feats.merge(train_scores, on='id', how='left')

In [21]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

train_feats['score_class'] = le.fit_transform(train_feats['score'])

In [22]:
target_col = ['score']

drop_cols = ['id', 'score_class']
train_cols = list()

train_cols = [col for col in train_feats.columns if col not in target_col + drop_cols]

In [23]:
nan_cols = train_feats.columns[train_feats.isna().any()].tolist()

# 결측치 최빈값으로 처리 
for col in nan_cols:
    mode_value_train = train_feats[col].mode()[0] #최빈값 여러 개 일 경우 첫 번째꺼 선택 
    train_feats[col].fillna(mode_value_train, inplace=True)
    
for col in test_feats.columns[test_feats.isna().any()].tolist():
    # Find the most frequent value in the training set for the current feature
    most_frequent_value_train = train_feats[col].mode()[0]
    
    # Fill missing values in the test set with the most frequent value from the training set
    test_feats[col].fillna(most_frequent_value_train, inplace=True)

train_feats.shape, test_feats.shape  

((2471, 659), (3, 657))

In [24]:
train_feats = train_feats.replace([np.inf], 1e10)
train_feats = train_feats.replace([-np.inf], -1e10)
test_feats = test_feats.replace([np.inf], 1e10)
test_feats = test_feats.replace([-np.inf], -1e10)

In [25]:
clean_memory()

# 1.3 Modeling

**Lgbm**

In [26]:
models_dict = {}
lgbm_scores = []

test_predict_lgb = []
best_params = {'boosting_type': 'gbdt', 
               'metric': 'rmse',
               'reg_alpha': 0.003188447814669599, 
               'reg_lambda': 0.0010228604507564066, 
               'colsample_bytree': 0.5420247656839267, 
               'subsample': 0.9778252382803456, 
               'feature_fraction': 0.8,
               'bagging_freq': 1,
               'bagging_fraction': 0.75,
               'learning_rate': 0.01716485155812008, 
               'num_leaves': 19, 
               'min_child_samples': 46,
               'verbosity': -1,
               'random_state': 42,
               'n_estimators': 500,
               }

for i in range(1): 
    kf = model_selection.KFold(n_splits=10, random_state=42 + i, shuffle=True)

    oof_valid_lgb = np.zeros(train_feats.shape[0], )

    X_test = test_feats[train_cols]


    for fold, (train_idx, valid_idx) in enumerate(kf.split(train_feats)):

        print("==-"* 50)
        print("Fold : ", fold)

        X_train, y_train = train_feats.iloc[train_idx][train_cols], train_feats.iloc[train_idx][target_col]
        X_valid, y_valid = train_feats.iloc[valid_idx][train_cols], train_feats.iloc[valid_idx][target_col]

        print("Trian :", X_train.shape, y_train.shape)
        print("Valid :", X_valid.shape, y_valid.shape)

        params = {
            "objective": "regression",
            "metric": "rmse",
            'random_state': 42,
            "n_estimators" : 12001,
            "verbosity": 0,
            "device_type": "cpu",
            **best_params
        }

        lgb_model = lgb.LGBMRegressor(**params)

        early_stopping_callback = lgb.early_stopping(200, first_metric_only=True, verbose=False)
        verbose_callback = lgb.callback.record_evaluation({})

        lgb_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],  
                  callbacks=[early_stopping_callback, verbose_callback],
        )

        valid_predict = lgb_model.predict(X_valid)
        oof_valid_lgb[valid_idx] = valid_predict

        test_predict = lgb_model.predict(X_test)
        test_predict_lgb.append(test_predict)

        lgbm_score = metrics.mean_squared_error(y_valid, valid_predict, squared=False)
        print("Fold RMSE Score : ", lgbm_score)

        models_dict[f'{fold}_{i}'] = lgb_model


    oof_score = metrics.mean_squared_error(train_feats[target_col], oof_valid_lgb, squared=False)
    lgbm_scores.append(oof_score)

==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-
Fold :  0
Trian : (2223, 656) (2223, 1)
Valid : (248, 656) (248, 1)
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=0.9778252382803456 will be ignored. Current value: bagging_fraction=0.75
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=0.5420247656839267 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Fold RMSE Score :  0.5757524553175648
==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-
Fold :  1
Trian : (2224, 656) (2224, 1)
Valid : (247, 656) (247, 1)
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=0.9778252382803456 will be ignored. Current value: bagging_fraction=0.75
[LightGBM

In [27]:
lgbm_rmse = np.mean(lgbm_scores)
print(lgbm_rmse)

0.6082148657675444


In [28]:
test_feats['score'] = np.mean(test_predict_lgb, axis=0)
lgb_pred = test_feats[['id', 'score']]
lgb_pred

,id,score
0,0000aaaa,1.475460
1,2222bbbb,1.437128
2,4444cccc,1.437601


**Randomforest**

In [29]:
from sklearn.ensemble import RandomForestRegressor

models_dict = {}
rf_scores = []

test_predict_rf = []
best_params = {
    'n_estimators': 256, 
    'max_features': 'sqrt', 
    'max_depth': 10,  
    'bootstrap': True,
    'random_state': 42,
}

for i in range(1): 
    kf = model_selection.KFold(n_splits=10, random_state=42, shuffle=True)

    oof_valid_rf = np.zeros(train_feats.shape[0], )

    X_test = test_feats[train_cols]

    for fold, (train_idx, valid_idx) in enumerate(kf.split(train_feats)):

        print("==-"* 50)
        print("Fold : ", fold)

        X_train, y_train = train_feats.iloc[train_idx][train_cols], train_feats.iloc[train_idx][target_col]
        X_valid, y_valid = train_feats.iloc[valid_idx][train_cols], train_feats.iloc[valid_idx][target_col]

        print("Train:", X_train.shape, y_train.shape)
        print("Valid:", X_valid.shape, y_valid.shape)

        rf_model = RandomForestRegressor(**best_params)

        rf_model.fit(X_train, y_train)

        valid_predict = rf_model.predict(X_valid)
        oof_valid_rf[valid_idx] = valid_predict

        test_predict = rf_model.predict(X_test)
        test_predict_rf.append(test_predict)

        rf_score = metrics.mean_squared_error(y_valid, valid_predict, squared=False)
        print("Fold RMSE Score : ", rf_score)

        models_dict[f'{fold}_{i}'] = rf_model

    oof_score = metrics.mean_squared_error(train_feats[target_col], oof_valid_rf, squared=False)
    rf_scores.append(oof_score)


==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-
Fold :  0
Train: (2223, 656) (2223, 1)
Valid: (248, 656) (248, 1)
Fold RMSE Score :  0.6042508326828393
==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-
Fold :  1
Train: (2224, 656) (2224, 1)
Valid: (247, 656) (247, 1)
Fold RMSE Score :  0.5499532418488657
==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-
Fold :  2
Train: (2224, 656) (2224, 1)
Valid: (247, 656) (247, 1)
Fold RMSE Score :  0.6980361863040841
==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-
Fold :  3
Train: (2224, 656) (2224, 1)
Valid: (247, 656) (247, 1)
Fold RMSE Score : 

In [30]:
rf_rmse = np.mean(rf_scores)
print(rf_rmse)

0.6383467774570079


In [31]:
test_feats['score'] = np.mean(test_predict_rf, axis=0)
rf_pred = test_feats[['id','score']]
rf_pred

,id,score
0,0000aaaa,1.877687
1,2222bbbb,1.882486
2,4444cccc,1.865905


**Lightautoml**

In [32]:
!pip install --no-index -U --find-links=/kaggle/input/lightautoml-038-dependecies lightautoml==0.3.8
!pip install --no-index -U --find-links=/kaggle/input/lightautoml-038-dependecies pandas==2.0.3

Looking in links: /kaggle/input/lightautoml-038-dependecies
Processing /kaggle/input/lightautoml-038-dependecies/lightautoml-0.3.8-py3-none-any.whl
Processing /kaggle/input/lightautoml-038-dependecies/AutoWoE-1.3.2-py3-none-any.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/cmaes-0.10.0-py3-none-any.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/joblib-1.2.0-py3-none-any.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/json2html-1.3.0.tar.gz (from lightautoml==0.3.8)
  Preparing metadata (setup.py) ... - \ done
Processing /kaggle/input/lightautoml-038-dependecies/lightgbm-3.2.1-py3-none-manylinux1_x86_64.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/poetry_core-1.8.1-py3-none-any.whl (from lig

In [33]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

In [34]:
N_THREADS = 4
N_FOLDS = 10
RANDOM_STATE = 42
TEST_SIZE = 0.2
VAL_SIZE = 0.2
TIMEOUT = 1000
ADVANCED_ROLES = False
USE_QNT = True
TASK = 'reg'
ALGOS_FOR_BLEND = ["lgb", "dense", "tabnet", "fttransformer", "autoint"]
USE_PLR = True
TRAIN_BS = 128
USE_FS = False
FEATURE_RATIO = 0.8

TARGET_NAME = 'score'
    
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [35]:
task = Task(TASK)

roles = {
    'target': TARGET_NAME,
}

In [36]:
def use_plr(USE_PLR):
    if USE_PLR:
        return "plr"
    else:
        return "cont"

In [37]:
algo = 'denselight'
automl = TabularAutoML(
    task = task, 
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    general_params = {"use_algos": [[algo]]},
    nn_params = {
        "n_epochs": 10, 
        "bs": TRAIN_BS, 
        "num_workers": 0, 
        "path_to_save": None, 
        "freeze_defaults": True,
        "cont_embedder": use_plr(USE_PLR),
    },
    nn_pipeline_params = {"use_qnt": USE_QNT, "use_te": False},
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE, 'advanced_roles': ADVANCED_ROLES},
)

oof_pred = automl.fit_predict(train_feats.drop(columns=['id','score_class'], axis=1), roles = roles, verbose = 3)
test_pred = automl.predict(test_feats)

[05:59:45] Stdout logging level is INFO3.
[05:59:45] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[05:59:45] Task: reg

[05:59:45] Start automl preset with listed constraints:
[05:59:45] - time: 1000.00 seconds
[05:59:45] - CPU: 4 cores
[05:59:45] - memory: 16 GB

[05:59:45] Train data shape: (2471, 657)

[05:59:45] Layer 1 train process start. Time left 999.49 secs
[05:59:47] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN_denselight_0 ...
[05:59:47] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_TorchNN_denselight_0 =====
[05:59:59] Epoch: 0, train loss: 0.798934817314148, val loss: 0.5294961929321289, val metric: -0.5299918055534363
[06:00:05] Epoch: 1, train loss: 0.41552042961120605, val loss: 0.3456771969795227, val metric: -0.34628579020500183
[06:00:12] Epoch: 2, train loss: 0.347106397151947, val loss: 0.34037894010543823, val metric: -0.3409070372581482
[06:00:19] Epoch: 3, train loss: 0.30299174785614014, val loss: 0.322

In [38]:
nn_rmse = mean_squared_error(train_feats['score'], oof_pred.data, squared=False)
print(nn_rmse)

0.6075930036823013


In [39]:
nn_pred = test_feats[['id']]
nn_pred['score'] = 0
nn_pred['score'] = test_pred.data
display(nn_pred)

,id,score
0,0000aaaa,0.455746
1,2222bbbb,0.188265
2,4444cccc,0.234182


# 1.4 Simple Blending

In [40]:
best_sc = 1
best_weights = None

for w1 in np.arange(0, 1.01, 0.001):
    for w2 in np.arange(0, 1.01 - w1, 0.001):
        w3 = 1 - w1 - w2
        
        combined_pred = w1 * oof_valid_lgb + w2 * oof_valid_rf + w3 * oof_pred.data.flatten()
        
        sc = metrics.mean_squared_error(
            train_feats['score'].values,
            combined_pred,
            squared=False
        )

        if sc < best_sc:
            best_sc = sc
            best_weights = (w1, w2, w3)

print('Best OOF score = {:.5f}'.format(best_sc))
print('Best Weights =', best_weights)


Best OOF score = 0.60038
Best Weights = (0.49, 0.0, 0.51)


In [41]:
W = [best_weights[0], best_weights[1], best_weights[2]]
print(W)

# Assuming test_pred1, test_pred2, and test_pred3 are the three sets of predictions
test_preds = (
     lgb_pred['score'].values* W[0] +
     rf_pred['score'].values* W[1] +
     test_pred.data.flatten() * W[2]
)


[0.49, 0.0, 0.51]


In [42]:
test_feats['score'] = test_preds
sub1 = test_feats[['id', 'score']]
display(sub1)

,id,score
0,0000aaaa,0.955406
1,2222bbbb,0.800208
2,4444cccc,0.823857


# Second NoteBook

# 2-1 Libraries

In [43]:
import gc
import pandas as pd
import polars as pl
import numpy as np
import re
import random
import os
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
import optuna
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from collections import Counter
from tqdm import tqdm
import warnings
import optuna
from collections import defaultdict
import torch
from torch import nn
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader, random_split
from scipy.stats import skew, kurtosis
warnings.filterwarnings("ignore")

**Polars FE & Helper Functions**

In [44]:
num_cols = ['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']
activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
text_changes = ['q', ' ', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']


def count_by_values(df, colname, values):
    fts = df.select(pl.col('id').unique(maintain_order=True))
    for i, value in enumerate(values):
        tmp_df = df.group_by('id').agg(pl.col(colname).is_in([value]).sum().alias(f'{colname}_{i}_cnt'))
        fts  = fts.join(tmp_df, on='id', how='left') 
    return fts


def dev_feats(df):
    
    print("< Count by values features >")
    
    feats = count_by_values(df, 'activity', activities)
    feats = feats.join(count_by_values(df, 'text_change', text_changes), on='id', how='left') 
    feats = feats.join(count_by_values(df, 'down_event', events), on='id', how='left') 
    feats = feats.join(count_by_values(df, 'up_event', events), on='id', how='left') 

    print("< Input words stats features >")

    temp = df.filter((~pl.col('text_change').str.contains('=>')) & (pl.col('text_change') != 'NoChange'))
    temp = temp.group_by('id').agg(pl.col('text_change').str.concat('').str.extract_all(r'q+'))
    temp = temp.with_columns(input_word_count = pl.col('text_change').list.lengths(),
                             input_word_length_mean = pl.col('text_change').apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_max = pl.col('text_change').apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_std = pl.col('text_change').apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_median = pl.col('text_change').apply(lambda x: np.median([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_skew = pl.col('text_change').apply(lambda x: skew([len(i) for i in x] if len(x) > 0 else 0)))
    temp = temp.drop('text_change')
    feats = feats.join(temp, on='id', how='left') 


    
    print("< Numerical columns features >")

    temp = df.group_by("id").agg(pl.sum('action_time').suffix('_sum'), pl.mean(num_cols).suffix('_mean'), pl.std(num_cols).suffix('_std'),
                                 pl.median(num_cols).suffix('_median'), pl.min(num_cols).suffix('_min'), pl.max(num_cols).suffix('_max'),
                                 pl.quantile(num_cols, 0.5).suffix('_quantile'))
    feats = feats.join(temp, on='id', how='left') 


    print("< Categorical columns features >")
    
    temp  = df.group_by("id").agg(pl.n_unique(['activity', 'down_event', 'up_event', 'text_change']))
    feats = feats.join(temp, on='id', how='left') 


    
    print("< Idle time features >")

    temp = df.with_columns(pl.col('up_time').shift().over('id').alias('up_time_lagged'))
    temp = temp.with_columns((abs(pl.col('down_time') - pl.col('up_time_lagged')) / 1000).fill_null(0).alias('time_diff'))
    temp = temp.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.group_by("id").agg(inter_key_largest_lantency = pl.max('time_diff'),
                                   inter_key_median_lantency = pl.median('time_diff'),
                                   mean_pause_time = pl.mean('time_diff'),
                                   std_pause_time = pl.std('time_diff'),
                                   total_pause_time = pl.sum('time_diff'),
                                   pauses_half_sec = pl.col('time_diff').filter((pl.col('time_diff') > 0.5) & (pl.col('time_diff') < 1)).count(),
                                   pauses_1_sec = pl.col('time_diff').filter((pl.col('time_diff') > 1) & (pl.col('time_diff') < 1.5)).count(),
                                   pauses_1_half_sec = pl.col('time_diff').filter((pl.col('time_diff') > 1.5) & (pl.col('time_diff') < 2)).count(),
                                   pauses_2_sec = pl.col('time_diff').filter((pl.col('time_diff') > 2) & (pl.col('time_diff') < 3)).count(),
                                   pauses_3_sec = pl.col('time_diff').filter(pl.col('time_diff') > 3).count(),)
    feats = feats.join(temp, on='id', how='left') 
    
    print("< P-bursts features >")

    temp = df.with_columns(pl.col('up_time').shift().over('id').alias('up_time_lagged'))
    temp = temp.with_columns((abs(pl.col('down_time') - pl.col('up_time_lagged')) / 1000).fill_null(0).alias('time_diff'))
    temp = temp.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.with_columns(pl.col('time_diff')<2)
    temp = temp.with_columns(pl.when(pl.col("time_diff") & pl.col("time_diff").is_last()).then(pl.count()).over(pl.col("time_diff").rle_id()).alias('P-bursts'))
    temp = temp.drop_nulls()
    temp = temp.group_by("id").agg(pl.mean('P-bursts').suffix('_mean'), pl.std('P-bursts').suffix('_std'), pl.count('P-bursts').suffix('_count'),
                                   pl.median('P-bursts').suffix('_median'), pl.max('P-bursts').suffix('_max'),
                                   pl.first('P-bursts').suffix('_first'), pl.last('P-bursts').suffix('_last'))
    feats = feats.join(temp, on='id', how='left') 


    print("< R-bursts features >")

    temp = df.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.with_columns(pl.col('activity').is_in(['Remove/Cut']))
    temp = temp.with_columns(pl.when(pl.col("activity") & pl.col("activity").is_last()).then(pl.count()).over(pl.col("activity").rle_id()).alias('R-bursts'))
    temp = temp.drop_nulls()
    temp = temp.group_by("id").agg(pl.mean('R-bursts').suffix('_mean'), pl.std('R-bursts').suffix('_std'), 
                                   pl.median('R-bursts').suffix('_median'), pl.max('R-bursts').suffix('_max'),
                                   pl.first('R-bursts').suffix('_first'), pl.last('R-bursts').suffix('_last'))
    feats = feats.join(temp, on='id', how='left')
    
    return feats

**Pandas FE & Helper Functions**

In [45]:
def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)

AGGREGATIONS = ['count', 'mean', 'min', 'max', 'first', 'last', q1, 'median', q3, 'sum']

def reconstruct_essay(currTextInput):
    essayText = ""
    for Input in currTextInput.values:
        if Input[0] == 'Replace':
            replaceTxt = Input[2].split(' => ')
            essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
            continue
        if Input[0] == 'Paste':
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
            continue
        if Input[0] == 'Remove/Cut':
            essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
            continue
        if "M" in Input[0]:
            croppedTxt = Input[0][10:]
            splitTxt = croppedTxt.split(' To ')
            valueArr = [item.split(', ') for item in splitTxt]
            moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))
            if moveData[0] != moveData[2]:
                if moveData[0] < moveData[2]:
                    essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                else:
                    essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
            continue
        essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
    return essayText


def get_essay_df(df):
    df       = df[df.activity != 'Nonproduction']
    temp     = df.groupby('id').apply(lambda x: reconstruct_essay(x[['activity', 'cursor_position', 'text_change']]))
    essay_df = pd.DataFrame({'id': df['id'].unique().tolist()})
    essay_df = essay_df.merge(temp.rename('essay'), on='id')
    return essay_df


def word_feats(df):
    essay_df = df
    df['word'] = df['essay'].apply(lambda x: re.split(' |\\n|\\.|\\?|\\!',x))
    df = df.explode('word')
    df['word_len'] = df['word'].apply(lambda x: len(x))
    df = df[df['word_len'] != 0]

    word_agg_df = df[['id','word_len']].groupby(['id']).agg(AGGREGATIONS)
    word_agg_df.columns = ['_'.join(x) for x in word_agg_df.columns]
    word_agg_df['id'] = word_agg_df.index
    word_agg_df = word_agg_df.reset_index(drop=True)
    return word_agg_df


def sent_feats(df):
    df['sent'] = df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
    df = df.explode('sent')
    df['sent'] = df['sent'].apply(lambda x: x.replace('\n','').strip())
    # Number of characters in sentences
    df['sent_len'] = df['sent'].apply(lambda x: len(x))
    # Number of words in sentences
    df['sent_word_count'] = df['sent'].apply(lambda x: len(x.split(' ')))
    df = df[df.sent_len!=0].reset_index(drop=True)

    sent_agg_df = pd.concat([df[['id','sent_len']].groupby(['id']).agg(AGGREGATIONS), 
                             df[['id','sent_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1)
    sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
    sent_agg_df['id'] = sent_agg_df.index
    sent_agg_df = sent_agg_df.reset_index(drop=True)
    sent_agg_df.drop(columns=["sent_word_count_count"], inplace=True)
    sent_agg_df = sent_agg_df.rename(columns={"sent_len_count":"sent_count"})
    return sent_agg_df


def parag_feats(df):
    df['paragraph'] = df['essay'].apply(lambda x: x.split('\n'))
    df = df.explode('paragraph')
    # Number of characters in paragraphs
    df['paragraph_len'] = df['paragraph'].apply(lambda x: len(x)) 
    # Number of words in paragraphs
    df['paragraph_word_count'] = df['paragraph'].apply(lambda x: len(x.split(' ')))
    df = df[df.paragraph_len!=0].reset_index(drop=True)
    
    paragraph_agg_df = pd.concat([df[['id','paragraph_len']].groupby(['id']).agg(AGGREGATIONS), 
                                  df[['id','paragraph_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1) 
    paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
    paragraph_agg_df['id'] = paragraph_agg_df.index
    paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
    paragraph_agg_df.drop(columns=["paragraph_word_count_count"], inplace=True)
    paragraph_agg_df = paragraph_agg_df.rename(columns={"paragraph_len_count":"paragraph_count"})
    return paragraph_agg_df

def product_to_keys(logs, essays):
    essays['product_len'] = essays.essay.str.len()
    tmp_df = logs[logs.activity.isin(['Input', 'Remove/Cut'])].groupby(['id']).agg({'activity': 'count'}).reset_index().rename(columns={'activity': 'keys_pressed'})
    essays = essays.merge(tmp_df, on='id', how='left')
    essays['product_to_keys'] = essays['product_len'] / essays['keys_pressed']
    return essays[['id', 'product_to_keys']]

def get_keys_pressed_per_second(logs):
    temp_df = logs[logs['activity'].isin(['Input', 'Remove/Cut'])].groupby(['id']).agg(keys_pressed=('event_id', 'count')).reset_index()
    temp_df_2 = logs.groupby(['id']).agg(min_down_time=('down_time', 'min'), max_up_time=('up_time', 'max')).reset_index()
    temp_df = temp_df.merge(temp_df_2, on='id', how='left')
    temp_df['keys_per_second'] = temp_df['keys_pressed'] / ((temp_df['max_up_time'] - temp_df['min_down_time']) / 1000)
    return temp_df[['id', 'keys_per_second']]


**Solution**

In [46]:
data_path     = '/kaggle/input/linking-writing-processes-to-writing-quality/'
train_logs    = pl.scan_csv(data_path + 'train_logs.csv')
train_feats   = dev_feats(train_logs)
train_feats   = train_feats.collect().to_pandas()

print('< Essay Reconstruction >')
train_logs             = train_logs.collect().to_pandas()
train_essays           = get_essay_df(train_logs)
train_feats            = train_feats.merge(word_feats(train_essays), on='id', how='left')
train_feats            = train_feats.merge(sent_feats(train_essays), on='id', how='left')
train_feats            = train_feats.merge(parag_feats(train_essays), on='id', how='left')
train_feats            = train_feats.merge(get_keys_pressed_per_second(train_logs), on='id', how='left')
train_feats            = train_feats.merge(product_to_keys(train_logs, train_essays), on='id', how='left')


print('< Mapping >')
train_scores   = pd.read_csv(data_path + 'train_scores.csv')
data           = train_feats.merge(train_scores, on='id', how='left')
x              = data.drop(['id', 'score'], axis=1)
y              = data['score'].values
print(f'Number of features: {len(x.columns)}')


print('< Testing Data >')
test_logs   = pl.scan_csv(data_path + 'test_logs.csv')
test_feats  = dev_feats(test_logs)
test_feats  = test_feats.collect().to_pandas()

test_logs             = test_logs.collect().to_pandas()
test_essays           = get_essay_df(test_logs)
test_feats            = test_feats.merge(word_feats(test_essays), on='id', how='left')
test_feats            = test_feats.merge(sent_feats(test_essays), on='id', how='left')
test_feats            = test_feats.merge(parag_feats(test_essays), on='id', how='left')
test_feats            = test_feats.merge(get_keys_pressed_per_second(test_logs), on='id', how='left')
test_feats            = test_feats.merge(product_to_keys(test_logs, test_essays), on='id', how='left')

test_ids = test_feats['id'].values
testin_x = test_feats.drop(['id'], axis=1)

# print('< Learning and Evaluation >')
# param = {'n_estimators': 1024,
#          'learning_rate': 0.005,
#          'metric': 'rmse',
#          'random_state': 42,
#          'force_col_wise': True,
#          'verbosity': 0,}
# solution = LGBMRegressor(**param)
# y_pred   = evaluate(x.copy(), y.copy(), solution, test_x=testin_x.copy()) 

# sub = pd.DataFrame({'id': test_ids, 'score': y_pred})
# sub.to_csv('submission.csv', index=False)

< Count by values features >
< Input words stats features >
< Numerical columns features >
< Categorical columns features >
< Idle time features >
< P-bursts features >
< R-bursts features >
< Essay Reconstruction >
< Mapping >
Number of features: 165
< Testing Data >
< Count by values features >
< Input words stats features >
< Numerical columns features >
< Categorical columns features >
< Idle time features >
< P-bursts features >
< R-bursts features >


In [47]:
nan_cols = train_feats.columns[train_feats.isna().any()].tolist()
for col in nan_cols:
    mode_value = train_feats[col].mode()[0]
    train_feats[col].fillna(mode_value, inplace=True)    

In [48]:
nan_cols = test_feats.columns[test_feats.isna().any()].tolist()
for col in nan_cols:
    mode_value = train_feats[col].mode()[0]
    test_feats[col].fillna(mode_value, inplace=True)

# 2.2 Optimized Weight

In [49]:
train_feats = train_feats.merge(train_scores, on='id', how='left')

In [50]:
rmse = lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False)

In [51]:
nan_cols = data.columns[data.isna().any()].tolist()
for col in nan_cols:
    mode_value = data[col].mode()[0]
    data[col].fillna(mode_value, inplace=True) 

In [52]:
nan_cols = testin_x.columns[testin_x.isna().any()].tolist()
for col in nan_cols:
    mode_value = data[col].mode()[0]
    testin_x[col].fillna(mode_value, inplace=True)

In [53]:
from sklearn.model_selection import KFold
n_splits = 10
kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)
data["fold"] = -1
for idx, (train_idx, val_idx) in enumerate(kf.split(data)):
    data.loc[val_idx, "fold"] = idx

In [54]:
from collections import defaultdict
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from catboost import CatBoostRegressor
import xgboost as xgb

drop_cols = ["id", "score", "fold"]
oof_df = pd.DataFrame()
models = defaultdict(list)
models_to_ensemble = ["xgboost", "lgbm",'catboost','randomforest', 'ElasticNet']
model_params = {
    "catboost": {
        "iterations": 5000,
        "early_stopping_rounds": 50,
        "depth": 6,
        "loss_function": "RMSE",
        "random_seed": 42,
        "silent": True
    },
    'ElasticNet':{
        'alpha':0.1,
        'l1_ratio':0.7,
        'max_iter':2500,
        'tol':1e-4,
    },
    "lgbm": {
        'n_estimators': 1024,
        'learning_rate': 0.005,
        'metric': 'rmse',
        'random_state': 42,
        'force_col_wise': True,
        'verbosity': 0
    },
    "xgboost": {
        "max_depth": 4,
        "learning_rate": 0.1,
        "objective": "reg:squarederror",
        "num_estimators": 1000,
        "num_boost_round": 1000,
        "eval_metric": "rmse",
        "seed": 42
    },
    'randomforest': {
        'max_depth': 5,
        'random_state': 42,
        'bootstrap': True,
        'n_estimators': 500,
        
    }
}
for idx, model_name in enumerate(models_to_ensemble):
    params = model_params[model_name]
    oof_folds = pd.DataFrame()
    print(f"Started the {model_name} model...")
    for fold in range(n_splits):
        if model_name == "lgbm":
            model = LGBMRegressor(**params)
        elif model_name == 'ElasticNet':
            model = ElasticNet(**params)    
        elif model_name == "xgboost":
            model = xgb.XGBRegressor(**params)
        elif model_name == "catboost":
            model = CatBoostRegressor(**params)
        elif model_name == 'randomforest':
            model = RandomForestRegressor(**params)
        else:
            raise ValueError("Unknown base model name.")

        x_train = data[data["fold"] != fold].reset_index(drop=True)
        x_valid = data[data["fold"] == fold].reset_index(drop=True)

        y_train = x_train["score"]
        y_valid = x_valid["score"]
        ids = x_valid["id"]

        x_train = x_train.drop(drop_cols, axis="columns")
        x_valid = x_valid.drop(drop_cols, axis="columns")
        model.fit(x_train, y_train)
        val_preds = model.predict(x_valid)
        oof_fold = pd.concat(
            [ids, y_valid, pd.Series(val_preds)], 
            axis=1).rename({0: f"{model_name}_preds"}, axis="columns")
        oof_folds = pd.concat([oof_folds, oof_fold])
        models[model_name].append(model)
        print(f"Fold: {fold} - Score: {rmse(oof_fold['score'], oof_fold[f'{model_name}_preds']):.5f}")
    
    if idx == 0:
        oof_df = pd.concat([oof_df, oof_folds])
    else:
        oof_df[f"{model_name}_preds"] = oof_folds[f"{model_name}_preds"]
    cv_score = rmse(oof_df["score"], oof_df[f"{model_name}_preds"])
    print(f"{model_name} cv_score: ", round(cv_score, 5))

Started the xgboost model...
Fold: 0 - Score: 0.56587
Fold: 1 - Score: 0.52446
Fold: 2 - Score: 0.69334
Fold: 3 - Score: 0.62192
Fold: 4 - Score: 0.60955
Fold: 5 - Score: 0.59880
Fold: 6 - Score: 0.66036
Fold: 7 - Score: 0.63783
Fold: 8 - Score: 0.64389
Fold: 9 - Score: 0.61004
xgboost cv_score:  0.61825
Started the lgbm model...
Fold: 0 - Score: 0.56053
Fold: 1 - Score: 0.52128
Fold: 2 - Score: 0.68022
Fold: 3 - Score: 0.61701
Fold: 4 - Score: 0.59999
Fold: 5 - Score: 0.59928
Fold: 6 - Score: 0.64646
Fold: 7 - Score: 0.64290
Fold: 8 - Score: 0.63973
Fold: 9 - Score: 0.59038
lgbm cv_score:  0.61133
Started the catboost model...
Fold: 0 - Score: 0.56762
Fold: 1 - Score: 0.52392
Fold: 2 - Score: 0.69578
Fold: 3 - Score: 0.60099
Fold: 4 - Score: 0.61504
Fold: 5 - Score: 0.60909
Fold: 6 - Score: 0.65262
Fold: 7 - Score: 0.63881
Fold: 8 - Score: 0.63642
Fold: 9 - Score: 0.59231
catboost cv_score:  0.61489
Started the randomforest model...
Fold: 0 - Score: 0.58704
Fold: 1 - Score: 0.53849
Fo

In [55]:
import numpy as np
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize

pred_cols = [f"{model_name}_preds" for model_name in models_to_ensemble]
true_targets = oof_df["score"]

def objective_function(weights):
    ensemble_preds = (oof_df[pred_cols] * weights).sum(axis=1)
    score = rmse(oof_df["score"], ensemble_preds)
    return score

def find_weights(oof_df):
    len_models = len(models_to_ensemble)
    initial_weights = np.ones(len_models) / len_models
    bounds = [(0, 1)] * len_models
    result = minimize(objective_function, initial_weights, bounds=bounds, method='SLSQP') # L-BFGS-B
    optimized_weights = result.x
    optimized_weights /= np.sum(optimized_weights)
    return optimized_weights

optimized_weights = find_weights(oof_df)
print("Optimized Weights:", optimized_weights)

Optimized Weights: [0.05471952 0.46892797 0.31513568 0.         0.16121683]


In [56]:
oof_df["ensemble_optimized_preds"] = (oof_df[pred_cols] * optimized_weights).sum(axis=1)
cv_optimized = rmse(oof_df["score"], oof_df["ensemble_optimized_preds"])
print("cv_score with optimized weights: ", round(cv_optimized, 5))

cv_score with optimized weights:  0.6055


In [57]:
oof_df.to_csv("oof_df.csv")

In [58]:
preds = None
for weights, model_name in zip(optimized_weights, models_to_ensemble):
    models_list = models[model_name]
    current_preds = weights * np.mean([model.predict(testin_x) for model in models_list], axis=0)
    if preds is None:
        preds = current_preds
    else:
        preds += current_preds

In [59]:
y_pred = preds
sub2 = pd.DataFrame({'id': test_ids, 'score': y_pred})


In [60]:
sub2

,id,score
0,0000aaaa,1.733327
1,2222bbbb,1.521469
2,4444cccc,1.508271


# Third NoteBook

# 3.1 Import Libraries 

In [61]:
import re, random, torch, pickle, gc, os, sklearn
import optuna
import sys
import lightgbm as lgb
import polars as pl
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
## Sklearn package
from sklearn.preprocessing import RobustScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold
from sklearn.metrics import mean_squared_error 
from scipy.stats import skew, kurtosis

import ctypes
libc = ctypes.CDLL("libc.so.6")  # clear the memory
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 99

In [62]:
DEBUG = True
SEED = 42
N_FOLD = 10

In [63]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
seed_everything(SEED)

In [64]:
data_path = '/kaggle/input/linking-writing-processes-to-writing-quality/'
train_logs = pl.scan_csv(data_path + 'train_logs.csv') # Create a datafrom using polars for fast execution
display(train_logs.collect().head()) # collect() produce the actual df
# Train_logs dataset has 11 columns

id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
str,i64,i64,i64,i64,str,str,str,str,i64,i64
"""001519c8""",1,4526,4557,31,"""Nonproduction""","""Leftclick""","""Leftclick""","""NoChange""",0,0
"""001519c8""",2,4558,4962,404,"""Nonproduction""","""Leftclick""","""Leftclick""","""NoChange""",0,0
"""001519c8""",3,106571,106571,0,"""Nonproduction""","""Shift""","""Shift""","""NoChange""",0,0
"""001519c8""",4,106686,106777,91,"""Input""","""q""","""q""","""q""",1,1
"""001519c8""",5,107196,107323,127,"""Input""","""q""","""q""","""q""",2,1


# 3-2. Feature Engineering

In [65]:
class FeatureExtractor():
    def __init__(self, logs):
        self.logs = logs # Training logs
        
    def count_by_values(self, colname, used_cols):
        fts = self.logs.select(pl.col('id').unique(maintain_order=True))
        for i, col in enumerate(used_cols):
            tmp_logs = self.logs.group_by('id').agg(
                            pl.col(colname).is_in([col]).sum().alias(f'{colname}_{i}_cnt')
                                    )
            fts  = fts.join(tmp_logs, on='id', how='left') 
        return fts
    
    # Create the features from statistics of activities, text changes, events
    def create_count_by_values_feats(self):
        activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
        events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.',
                       ',', 'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
        text_changes = ['q', ' ', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']        
        #=== Create the feature columns using count by values ===
        df = self.count_by_values('activity', activities) # Create 'activity' column
        df = df.join(self.count_by_values('text_change', text_changes), on='id', how='left') 
        df = df.join(self.count_by_values('down_event', events), on='id', how='left') 
        df = df.join(self.count_by_values('up_event', events), on='id', how='left')
        # print(df.collect().head())
        return df
    
    # Create the features 
    def create_input_words_feats(self):
        # Filter no changes 
        df = self.logs.filter((~pl.col('text_change').str.contains('=>')) & (pl.col('text_change') != 'NoChange'))
        # Aggregate the text changes by id
        df = df.group_by('id').agg(pl.col('text_change').str.concat('').str.extract_all(r'q+'))
        # creates only two columns ('id' and 'text_change') 
        df = df.with_columns(input_word_count=pl.col('text_change').list.lengths(),
                             input_word_length_mean=pl.col('text_change').apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_max=pl.col('text_change').apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_std=pl.col('text_change').apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_median=pl.col('text_change').apply(lambda x: np.median([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_skew=pl.col('text_change').apply(lambda x: skew([len(i) for i in x] if len(x) > 0 else 0)))
        df = df.drop('text_change') # Remove 'text_change' to avoid including duplicated `text_change` column
        return df
    
    # Create the statistical numeric features (e.g. sum, median, mean min, 0.5_quantile)
    def create_numeric_feats(self):
        num_cols = ['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']
        df = self.logs.group_by("id").agg(pl.sum('action_time').suffix('_sum'),
                                                pl.mean(num_cols).suffix('_mean'),
                                                pl.std(num_cols).suffix('_std'),
                                                pl.median(num_cols).suffix('_median'), pl.min(num_cols).suffix('_min'), pl.max(num_cols).suffix('_max'),
                                                pl.quantile(num_cols, 0.5).suffix('_quantile'))
        return df
    
    def create_categorical_feats(self):
        df  = self.logs.group_by("id").agg(
                pl.n_unique(['activity', 'down_event', 'up_event', 'text_change']))
        return df
    
    # Create the idle time features 
    def create_idle_time_feats(self):
        df = self.logs.with_columns(pl.col('up_time').shift().over('id').alias('up_time_lagged'))
        df = df.with_columns((abs(pl.col('down_time') - pl.col('up_time_lagged')) / 1000).fill_null(0).alias('time_diff'))
        df = df.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
        df = df.group_by("id").agg(inter_key_largest_lantency = pl.max('time_diff'),
                                   inter_key_median_lantency = pl.median('time_diff'),
                                   mean_pause_time = pl.mean('time_diff'),
                                   std_pause_time = pl.std('time_diff'),
                                   total_pause_time = pl.sum('time_diff'),
                                   pauses_half_sec = pl.col('time_diff').filter((pl.col('time_diff') > 0.5) & (pl.col('time_diff') < 1)).count(),
                                   pauses_1_sec = pl.col('time_diff').filter((pl.col('time_diff') > 1) & (pl.col('time_diff') < 1.5)).count(),
                                   pauses_1_half_sec = pl.col('time_diff').filter((pl.col('time_diff') > 1.5) & (pl.col('time_diff') < 2)).count(),
                                   pauses_2_sec = pl.col('time_diff').filter((pl.col('time_diff') > 2) & (pl.col('time_diff') < 3)).count(),
                                   pauses_3_sec = pl.col('time_diff').filter(pl.col('time_diff') > 3).count(),)
        return df
    
    # Create p-bursts features using up and down time and activity
    def create_p_bursts_feats(self):
        df = self.logs.with_columns(pl.col('up_time').shift().over('id').alias('up_time_lagged'))
        df = df.with_columns((abs(pl.col('down_time') - pl.col('up_time_lagged')) / 1000).fill_null(0).alias('time_diff'))
        df = df.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
        df = df.with_columns(pl.col('time_diff')<2)
        df = df.with_columns(pl.when(pl.col("time_diff") & pl.col("time_diff").is_last()).then(pl.count()).over(pl.col("time_diff").rle_id()).alias('P-bursts'))
        df = df.drop_nulls()
        df = df.group_by("id").agg(pl.mean('P-bursts').suffix('_mean'),
                                   pl.std('P-bursts').suffix('_std'),
                                   pl.count('P-bursts').suffix('_count'),
                                   pl.median('P-bursts').suffix('_median'),
                                   pl.max('P-bursts').suffix('_max'),
                                   pl.first('P-bursts').suffix('_first'),
                                   pl.last('P-bursts').suffix('_last'))
        return df
    
    # Create r-burst features using activity 
    def create_r_bursts_feats(self):
        df = self.logs.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
        df = df.with_columns(pl.col('activity').is_in(['Remove/Cut']))
        df = df.with_columns(pl.when(pl.col("activity") & pl.col("activity").is_last()).then(pl.count()).over(pl.col("activity").rle_id()).alias('R-bursts'))
        df = df.drop_nulls()
        df = df.group_by("id").agg(pl.mean('R-bursts').suffix('_mean'),
                                   pl.std('R-bursts').suffix('_std'), 
                                   pl.median('R-bursts').suffix('_median'),
                                   pl.max('R-bursts').suffix('_max'),
                                   pl.first('R-bursts').suffix('_first'),
                                   pl.last('R-bursts').suffix('_last'))
        return df

    # Main function creates all 122 features
    def create_feats(self):
        feats = self.create_count_by_values_feats()  # 52 columns in total
#         print(f"< Count by values features > {len(feats.columns)}")        
        feats = feats.join(self.create_input_words_feats(), on='id', how='left')  # 58 columns
#         print(f"< Input words stats features > {len(feats.columns)}")
        feats = feats.join(self.create_numeric_feats(), on='id', how='left') # 89 columns
#         print(f"< Numerical features > {len(feats.columns)}")
        feats = feats.join(self.create_categorical_feats(), on='id', how='left') # 93 columns      
#         print(f"< Categorical features > {len(feats.columns)}")
        feats = feats.join(self.create_idle_time_feats(), on='id', how='left') # 103 columns
#         print(f"< Idle time features > {len(feats.columns)}")
        feats = feats.join(self.create_p_bursts_feats(), on='id', how='left') # 110 columns
#         print(f"< P-bursts features > {len(feats.columns)}")
        feats = feats.join(self.create_r_bursts_feats() , on='id', how='left') # 116 columns
#         print(f"< R-bursts features > {len(feats.columns)}")        
        return feats # 116 features

In [66]:
fe = FeatureExtractor(train_logs)
train_feats = fe.create_feats() # Extract features from trainning logs (polars df)
train_feats = train_feats.collect().to_pandas() # Convert polars df to pandas df
train_feats.to_csv("train_feats_0.csv")
# print(train_feats.describe())
train_logs = train_logs.collect().to_pandas()  # Convert polars df to pandas df
del fe

In [67]:
def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)

class EssayConstructor():
    def __init__(self, logs):
        self.logs = logs
        self.train_essays = self.get_train_essays(self.logs)
        self.AGGREGATIONS = ['count', 'mean', 'min', 'max', 'first', 'last', q1, 'median', q3, 'sum']
    
    # Get the essay from train logs 
    def get_train_essays(self, logs):
        def reconstruct_essay(currTextInput):
            essayText = ""
            for Input in currTextInput.values:
                if Input[0] == 'Replace':
                    replaceTxt = Input[2].split(' => ')
                    essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                    continue
                if Input[0] == 'Paste':
                    essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                    continue
                if Input[0] == 'Remove/Cut':
                    essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                    continue
                if "M" in Input[0]:
                    croppedTxt = Input[0][10:]
                    splitTxt = croppedTxt.split(' To ')
                    valueArr = [item.split(', ') for item in splitTxt]
                    moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))
                    if moveData[0] != moveData[2]:
                        if moveData[0] < moveData[2]:
                            essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                        else:
                            essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
                    continue
                essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
            return essayText
        
        # Filter logs 
        df = logs[logs.activity != 'Nonproduction']
        group_df = df.groupby('id').apply(lambda x: reconstruct_essay(x[['activity', 'cursor_position', 'text_change']]))
        essay_df = pd.DataFrame({'id': df['id'].unique().tolist()})
        essay_df = essay_df.merge(group_df.rename('essay'), on='id')
        return essay_df

    # Create word level features from train essay
    def create_word_feats(self):
        df = self.train_essays.copy()
        df['word'] = df['essay'].apply(lambda x: re.split(' |\\n|\\.|\\?|\\!',x))
        df = df.explode('word')
        df['word_len'] = df['word'].apply(lambda x: len(x))
        df = df[df['word_len'] != 0] # Remove all the no-word record
        # Aggregate word level features
        word_agg_df = df[['id','word_len']].groupby(['id']).agg(self.AGGREGATIONS)
        word_agg_df.columns = ['_'.join(x) for x in word_agg_df.columns]
        word_agg_df['id'] = word_agg_df.index
        word_agg_df = word_agg_df.reset_index(drop=True)
        return word_agg_df
    # Create sentence level features
    def create_sentence_feats(self):
        df = self.train_essays.copy()
        df['sent'] = df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
        df = df.explode('sent')
        df['sent'] = df['sent'].apply(lambda x: x.replace('\n','').strip())
        # Number of characters in sentences
        df['sent_len'] = df['sent'].apply(lambda x: len(x))
        # Number of words in sentences
        df['sent_word_count'] = df['sent'].apply(lambda x: len(x.split(' ')))
        df = df[df.sent_len!=0].reset_index(drop=True)
        # Aggregate sentence level features
        sent_agg_df = pd.concat([df[['id','sent_len']].groupby(['id']).agg(self.AGGREGATIONS), 
                                 df[['id','sent_word_count']].groupby(['id']).agg(self.AGGREGATIONS)], axis=1)
        sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
        sent_agg_df['id'] = sent_agg_df.index
        sent_agg_df = sent_agg_df.reset_index(drop=True)
        sent_agg_df.drop(columns=["sent_word_count_count"], inplace=True)
        sent_agg_df = sent_agg_df.rename(columns={"sent_len_count":"sent_count"})
        return sent_agg_df
    # Create paragraph level features
    def create_paragraph_feats(self):
        df = self.train_essays.copy()
        df['paragraph'] = df['essay'].apply(lambda x: x.split('\n'))
        df = df.explode('paragraph')
        # Number of characters in paragraphs
        df['paragraph_len'] = df['paragraph'].apply(lambda x: len(x)) 
        # Number of words in paragraphs
        df['paragraph_word_count'] = df['paragraph'].apply(lambda x: len(x.split(' ')))
        df = df[df.paragraph_len!=0].reset_index(drop=True)
        # Aggregate paragraph level features
        paragraph_agg_df = pd.concat([df[['id','paragraph_len']].groupby(['id']).agg(self.AGGREGATIONS), 
                                      df[['id','paragraph_word_count']].groupby(['id']).agg(self.AGGREGATIONS)], axis=1) 
        paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
        paragraph_agg_df['id'] = paragraph_agg_df.index
        paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
        paragraph_agg_df.drop(columns=["paragraph_word_count_count"], inplace=True)
        paragraph_agg_df = paragraph_agg_df.rename(columns={"paragraph_len_count":"paragraph_count"})
        return paragraph_agg_df

    
    # Create product to keys features
    def create_product_to_keys_feats(self):
        essays = self.train_essays.copy()
        logs = self.logs.copy()
        essays['product_len'] = essays['essay'].str.len()
        tmp_df = logs[logs['activity'].isin(['Input', 'Remove/Cut'])].groupby(['id']).agg({'activity': 'count'}).reset_index().rename(columns={'activity': 'keys_pressed'})
        essays = essays.merge(tmp_df, on='id', how='left')
        essays['product_to_keys'] = essays['product_len'] / essays['keys_pressed']
        return essays[['id', 'product_to_keys']]

    # Create key pressed features
    def create_keys_pressed_feats(self):
        logs = self.logs.copy()
        temp_df = logs[logs['activity'].isin(['Input', 'Remove/Cut'])].groupby(['id']).agg(keys_pressed=('event_id', 'count')).reset_index()
        temp_df_2 = logs.groupby(['id']).agg(min_down_time=('down_time', 'min'), max_up_time=('up_time', 'max')).reset_index()
        temp_df = temp_df.merge(temp_df_2, on='id', how='left')
        temp_df['keys_per_second'] = temp_df['keys_pressed'] / ((temp_df['max_up_time'] - temp_df['min_down_time']) / 1000)
        return temp_df[['id', 'keys_per_second']]
    
    def create_feats(self, feats):
        feats = feats.merge(self.create_word_feats(), on='id', how='left') # 126 columns in total
#         print(f"{len(feats.columns)}")
        feats = feats.merge(self.create_sentence_feats(), on='id', how='left') # 145 columns
#         print(f"{len(feats.columns)}")
        feats = feats.merge(self.create_paragraph_feats(), on='id', how='left') # 164 columns
#         print(f"{len(feats.columns)}")
        feats = feats.merge(self.create_keys_pressed_feats(), on='id', how='left') # 166 columns
#         print(f"{len(feats.columns)}")
        feats = feats.merge(self.create_product_to_keys_feats(), on='id', how='left') # 165 columns
#         print(f"{len(feats.columns)}")
        return feats

In [68]:
print('< Essay Reconstruction >')
ec = EssayConstructor(train_logs)
train_feats = ec.create_feats(train_feats)
# Writing to csg
train_feats.to_csv("train_feats.csv")
del ec

< Essay Reconstruction >


In [69]:
train_scores  = pd.read_csv(data_path + 'train_scores.csv')
# Merge train features and train scores as training data
train_data = train_feats.merge(train_scores, on='id', how='left')

**null 값 채우기**

In [70]:
nan_cols = train_data.columns[train_data.isna().any()].tolist()
for col in nan_cols:
    mode_value = train_data[col].mode()[0]
    train_data[col].fillna(mode_value, inplace=True)    

In [71]:
print('< Testing Data >')  # Load test data
test_logs = pl.scan_csv(data_path + 'test_logs.csv')
# Extract statistical features
fe = FeatureExtractor(test_logs)
test_feats = fe.create_feats() 
test_feats = test_feats.collect().to_pandas()
test_logs = test_logs.collect().to_pandas()
# Extract essay features
ec = EssayConstructor(test_logs)
test_feats = ec.create_feats(test_feats)
print("Nan columns of test data", test_feats.columns[test_feats.isna().any()].tolist()) # Check if any na in the test data


# Split train data into X and Y
data_X = train_data.drop(['id', 'score'], axis=1)
data_Y = train_data['score'].values

< Testing Data >
Nan columns of test data ['P-bursts_mean', 'P-bursts_std', 'P-bursts_count', 'P-bursts_median', 'P-bursts_max', 'P-bursts_first', 'P-bursts_last', 'R-bursts_mean', 'R-bursts_std', 'R-bursts_median', 'R-bursts_max', 'R-bursts_first', 'R-bursts_last', 'word_len_count', 'word_len_mean', 'word_len_min', 'word_len_max', 'word_len_first', 'word_len_last', 'word_len_q1', 'word_len_median', 'word_len_q3', 'word_len_sum', 'sent_count', 'sent_len_mean', 'sent_len_min', 'sent_len_max', 'sent_len_first', 'sent_len_last', 'sent_len_q1', 'sent_len_median', 'sent_len_q3', 'sent_len_sum', 'sent_word_count_mean', 'sent_word_count_min', 'sent_word_count_max', 'sent_word_count_first', 'sent_word_count_last', 'sent_word_count_q1', 'sent_word_count_median', 'sent_word_count_q3', 'sent_word_count_sum']


In [72]:
nan_cols = test_feats.columns[test_feats.isna().any()].tolist()
for col in nan_cols:
    mode_value = train_data[col].mode()[0]
    test_feats[col].fillna(mode_value, inplace=True) 

In [73]:
test_ids = test_feats['id'].values
test_x = test_feats.drop(['id'], axis=1)

In [74]:
class ModelTrainer():
    def __init__(self, model_name, **params):
        # Model
        self.model_name = model_name
        self.params = params
        self.create_model()
        
        self.X = data_X
        self.Y = data_Y        
        print(f'Number of features: {len(self.X.columns)}')
        
    
    def make_pipeline(self, model):
        return Pipeline([
            ('remove_infs', FunctionTransformer(lambda x: np.nan_to_num(x, nan=np.nan, posinf=0, neginf=0))),
            ('imputer', SimpleImputer(strategy='mean')),
            ('normalizer', FunctionTransformer(lambda x: np.log1p(np.abs(x)))),
            ('scaler', RobustScaler()),
            ('model', model)
        ])
    
    # Create the model
    def create_model(self):
        match model_name:
            case "lgbm":
                self.model = LGBMRegressor(**self.params)
            case "xgb": 
                self.model = XGBRegressor(**self.params)
            case "catboost":
                self.model = CatBoostRegressor(**self.params)
            case "svr":
                self.model = self.make_pipeline(SVR(**self.params))
            case other:
                print("Not implemented")
                sys.exit(-1)
    
    # Get the trained model        
    def get_model(self):
        return self.model
        
    # Train the model with 10-fold CV
    def train_model(self):        
        early_stopping_callback = lgb.early_stopping(200, first_metric_only=True, verbose=False)
        verbose_callback = lgb.log_evaluation(100)        
        # Split the training data into 5 fold
        kf = KFold(n_splits=10, random_state=SEED, shuffle=True)
        fold_rmses = []
        for fold, (train_index, valid_index) in enumerate(kf.split(self.X)):
            train_x = self.X.iloc[train_index]
            train_y = self.Y[train_index]
            valid_x = self.X.iloc[valid_index]
            valid_y = self.Y[valid_index]
            # Fit the model with train x and train y
            self.model.fit(train_x, train_y)            
            predictions = self.model.predict(valid_x)
            rmse = mean_squared_error(y_true=valid_y, y_pred=predictions, squared=False) # Return RMSE
            fold_rmses.append(rmse)
        avg_rmse = np.mean(fold_rmses)
        print(f"Average rmse: {avg_rmse}") 
        return avg_rmse
    
    # Evaluate the model with entire X data
    def evaluation(self):
        preds = self.predict(self.X)
        rmse = mean_squared_error(y_true=self.Y, y_pred=preds, squared=False)
        return rmse
        
    # Predict the test data. 
    def predict(self, test_x):
        # Prediction loop
        tests_y = np.zeros((len(test_x), N_FOLD))
        for fold in range(N_FOLD):
            preds = self.model.predict(test_x)
            tests_y[:, fold] = preds
            #print(f"Fold = {fold} Prediction = {preds[:5]}")
        test_y = np.mean(tests_y, axis=1)
        return test_y# Average the prediction of each fold model
    
    # Clear the memory
    def clear_memory(self):
        del self.model
        libc.malloc_trim(0)
        torch.cuda.empty_cache()
        gc.collect()

# 3-3. VotingRegressor(by weight)

In [75]:
import torch
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import StandardScaler

**MLP**

class config:
    tp = 0.8
    lr = 1e-4
    n_epochs = 5
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class WPDataset(Dataset):
    
    def __init__(self, in_data, out_data, phase):
        super(WPDataset, self).__init__()
        
        self.phase = phase
        self.targets = out_data
        self.features = in_data
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        X = torch.tensor(self.features[index], dtype=torch.float32)
        
        if self.phase == "train":
            y = torch.tensor(self.targets[index], dtype=torch.float32)
            return X, y
        
        if self.phase == "test":
            return X

class NNModel(nn.Module):
    
    def __init__(self):
        super(NNModel, self).__init__()
        self.MLP = nn.Sequential(nn.Linear(165, 256),
                                 nn.LayerNorm(256),
                                 nn.ReLU(),
                                 nn.Linear(256,128),
                                 nn.LayerNorm(128),
                                 nn.ReLU(),
                                 nn.Linear(128,64),
                                 nn.LayerNorm(64),
                                 nn.ReLU(),
                                 nn.Linear(64, 1),)
    
    def forward(self, x):
        return self.MLP(x)

class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()

    def forward(self,x,y):
        mse_criterion = nn.MSELoss()
        loss = torch.sqrt(mse_criterion(x, y) + 1e-6)
        return loss

def learn(data, test_data):
    data_y = data['score'].values
    data_x = data.drop(['id', 'score'], axis=1)

    test_ids = test_data['id'].values
    x_test   = test_data.drop(['id'], axis=1)

    kf      = KFold(n_splits=10, random_state=42, shuffle=True)
    predicts = np.zeros((len(x_test), 10)) 
    oof_val = np.zeros((len(data_x), 1))
    
    # 변경된 부분: 모델 이름 변경
    for i, (train_index, valid_index) in enumerate(kf.split(data)):
        x_train = data_x.iloc[train_index]
        y_train = data_y[train_index]
        x_valid = data_x.iloc[valid_index]
        y_valid = data_y[valid_index]

        scaler  = StandardScaler()
        x_train = scaler.fit_transform(x_train)
        x_valid = scaler.transform(x_valid)
        x_test  = scaler.transform(x_test)
               
        train_ds = WPDataset(in_data=x_train, out_data=y_train, phase="train")
        valid_ds = WPDataset(in_data=x_valid, out_data=y_valid, phase="train")
        test_ds  = WPDataset(in_data=x_test, out_data=None, phase="test")

        train_dataloader = DataLoader(train_ds, batch_size=1, shuffle=True)
        valid_dataloader = DataLoader(valid_ds, batch_size=1, shuffle=False)
        test_dataloader  = DataLoader(test_ds, batch_size=1, shuffle=False)

        # 변경된 부분: 모델 이름 변경
        mlp_model = NNModel().to(config.device)
        optimizer = torch.optim.Adam(mlp_model.parameters(), lr=config.lr)
        scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=2)
        criterion = RMSELoss()
        n_epochs  = config.n_epochs
    
        train_loss_history=[]
        val_loss_history=[]
    
        count=0
        for epoch in range(n_epochs):
            print("EPOCH ", epoch+1)
            ## TRAIN PHASE :
            mlp_model.train()
            losses = []
            for fetres, labels in tqdm(train_dataloader):
                fetres, labels = fetres.to(config.device), labels.to(config.device)
                optimizer.zero_grad()
                preds = mlp_model(fetres)
                loss  = criterion(preds, labels)
                losses.append(loss.item()) 
                loss.backward()
                optimizer.step()
    
            avg_loss = np.mean(losses)
            train_loss_history.append(avg_loss)
            if count%2==0:
                print(f"Train Loss : {avg_loss}")
            
            ## VALIDATION PHASE : 
            mlp_model.eval()
            losses = []
            val_result = np.array([])
            for fetres, labels in valid_dataloader:
                fetres, labels = fetres.to(config.device), labels.to(config.device)
                preds = mlp_model(fetres)
                loss  = criterion(preds, labels)
                losses.append(loss.item())
                val_result = np.append(val_result, preds.detach().cpu().numpy())
                
            avg_loss = np.mean(losses)
            val_loss_history.append(avg_loss)
            if count%2==0:
                print(f"Valid Loss : {avg_loss}")
            
            scheduler.step(avg_loss)
            count+=1
            
            oof_val[valid_index, 0] = val_result
            
        ## Prediction
        mlp_model.eval()
        result = np.array([])
        for _, fetres in tqdm(enumerate(test_dataloader)):
            fetres = fetres.to(config.device)
            output = mlp_model(fetres)
            result = np.append(result, output.detach().cpu().numpy())
            
        predicts[:, i] = result 

    # 변경된 부분: 모델 반환
    return test_ids, predicts.mean(axis=1), oof_val, mlp_model


ids, predictions, val_pred, mlp_model= learn(train_data, test_feats)
mlp = pd.DataFrame(data={"id" : ids, "score" : predictions})

from sklearn.metrics import mean_squared_error
mlp_rmse = mean_squared_error(train_data['score'], val_pred, squared=False)
print('rmse:', mlp_rmse)

In [76]:
params_dict ={}
# CatBoostRegressor
params_dict['catboost'] =  {
    "iterations": 5000,
    "early_stopping_rounds": 50,
    "depth": 6,
    "loss_function": "RMSE",
    "random_seed": SEED,
    "silent": True
}
# Best parameters of LGBM
params_dict['lgbm'] = {
    'n_estimators': 1024,
    'learning_rate': 0.005,
    'metric': 'rmse',
    'random_state': SEED,
    'force_col_wise': True,
    'verbosity': 0,
}
# XGBRegressor
params_dict['xgb'] = {
     "max_depth": 4,
        "learning_rate": 0.1,
        "objective": "reg:squarederror",
        "num_estimators": 1000,
        "num_boost_round": 1000,
        "eval_metric": "rmse",
        "seed": SEED
}
# svr
params_dict['svr'] = {
    'kernel':'rbf',
    'C':1.0,
    'epsilon': 0.1
}

best_score = 1.0
# Find the optimal learning rate
def objective(trial, model_name):
    global params_dict
    # Parameters
    params = params_dict[model_name] # Load the default parameters
    params['learning_rate'] = trial.suggest_loguniform('learning_rate', 1e-4, 0.1)
    #         'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
    #         'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
    #         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
    #         'subsample': trial.suggest_float('subsample', 0.5, 1),
    #         'num_leaves' : trial.suggest_int('num_leaves', 8, 64),
    #         'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
    # Experiment the parameters
    trainer = ModelTrainer(model_name, **params)
    avg_score = trainer.train_model()
    # Save the model is the avg score < current best score
    global best_score
    if avg_score < best_score:
        best_score = avg_score
    # Clean up
    trainer.clear_memory()
    del trainer    
    print(f"Average result {avg_score} and the best score {best_score}")
    return avg_score

def run_optuna(model_name):
    study_name = f"{model_name}_study"
    study_file_path = f"/kaggle/working/{study_name}.db"
    if os.path.exists(study_file_path):
        os.remove(study_file_path)
    # # Create a study to find the optimal hyper-parameters    
    study = optuna.create_study(direction="minimize", study_name=study_name,
                                storage="sqlite:///" + f"{study_file_path}", # Storage path of the database keeping the study results
                                load_if_exists=False)  # Resume the existing study
    # Set up the timeout to avoid runing out of quote
    # n_jobs =-1 is CPU bounded
    study.optimize(lambda trial: objective(trial, model_name), 
                   n_jobs=1, n_trials=500,
                   show_progress_bar=True, gc_after_trial=True)
    ## Print the best parameters    
    best_trial = study.best_trial
    best_params = study.best_params
    # Print out the experiment results
    print(f"Best parameters: {best_params}\n\n"
          f"Number of finished trials: {len(study.trials)}\n\n"
          f"Best trial:{best_trial}")    
    return study

# Train the model and make the predictions
def train_model(model_name, is_loaded=True):
    best_params = None
    # If is_loaded is True, load the best parameters.
    # Otherwise, initiate an Optuna study to optimize parameters.
    if is_loaded:  # Loaded the best parameters that are found from previous experiments
        best_params = params_dict[model_name]
    else:
        study = run_optuna(model_name)
        ## Print the best parameters    
        best_trial = study.best_trial
        best_params = study.best_params
        # Print out the experiment results
        print(f"Best parameters: {best_params}\n\n"
              f"Number of finished trials: {len(study.trials)}\n\n"
              f"Best trial:{best_trial}")
    ## Parameters for LGBMRegressor model
    trainer = ModelTrainer(model_name, **best_params)
    trainer.train_model()
    rmse = trainer.evaluation()
    model = trainer.get_model()
    print(f"Complete training {model_name} RMSE = {rmse}")
    return model

# Collect all the models
models = []
model_names = ['svr', 'catboost', 'lgbm', 'xgb'] # Two models
preds_y = []
tests_y = []
for model_name in model_names:
    is_loaded = True
    model = train_model(model_name, is_loaded)
    models.append((model_name, model))
print(models)

def evaluate_models(models, data_X, data_Y):
    # split the full train data (data_X and data_Y) into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(data_X, data_Y,
                                                      test_size=0.2, random_state=SEED)

    # fit and evaluate the models
    weights = list()
    for name, model in models:
        # fit the model
        model.fit(X_train, y_train)
        # evaluate the model
        y_preds = model.predict(X_val)
        # Calculate the 
        rmse = mean_squared_error(y_true=y_val, y_pred=y_preds, squared=False)
        # store the performance
        weights.append(rmse)
    # report model performance
    return weights

try:
    weights = evaluate_models(models, data_X, data_Y)
    # Use the weights (scores) as a weighting for the ensemble
    ensemble = VotingRegressor(estimators=models, weights=weights)
    ensemble.fit(data_X, data_Y)
    test_y = ensemble.predict(test_x)
    print(test_y)
except Exception as e: 
    print(e)

Number of features: 165
Average rmse: 0.6349865670461656
Complete training svr RMSE = 0.4922363979317905
Number of features: 165
Average rmse: 0.6132598693017614
Complete training catboost RMSE = 0.22078497597052948
Number of features: 165
Average rmse: 0.6097788926296298
Complete training lgbm RMSE = 0.36245561024977546
Number of features: 165
Average rmse: 0.6166053958876445
Complete training xgb RMSE = 0.4118836814521802
[('svr', Pipeline(steps=[('remove_infs',
                 FunctionTransformer(func=<function ModelTrainer.make_pipeline.<locals>.<lambda> at 0x786ad997f010>)),
                ('imputer', SimpleImputer()),
                ('normalizer',
                 FunctionTransformer(func=<function ModelTrainer.make_pipeline.<locals>.<lambda> at 0x786ad997f130>)),
                ('scaler', RobustScaler()), ('model', SVR())])), ('catboost', <catboost.core.CatBoostRegressor object at 0x786ae1450490>), ('lgbm', LGBMRegressor(force_col_wise=True, learning_rate=0.005, metric='rmse

In [77]:
test_feats['score'] = test_y
sub3 = test_feats[['id','score']]
display(sub3)

,id,score
0,0000aaaa,1.943156
1,2222bbbb,1.728017
2,4444cccc,1.732300


# 4. Total Bledning

In [78]:
sub1.rename(columns={'score': 'score_1'}, inplace=True)
sub2.rename(columns={'score': 'score_2'}, inplace=True)
sub3.rename(columns={'score': 'score_3'}, inplace=True)

submission = pd.merge(sub1, sub2, on='id')
submission = pd.merge(submission, sub3, on='id')
submission['score'] = ((submission['score_1'] * (0.2)) + 
                       (submission['score_2'] * (0.5)) +
                       (submission['score_3'] * (0.3)))
                        

submission_final = submission[['id', 'score']]

In [79]:
submission_final.to_csv('submission.csv', index=False)

In [80]:
submission_final

,id,score
0,0000aaaa,1.640692
1,2222bbbb,1.439181
2,4444cccc,1.438597
